<a href="https://colab.research.google.com/github/nebulous05/influence-functions-pytorch/blob/main/influence-functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
### PROCEDURE ###

# Step 1: Train a model on the 10-class MNIST dataset
# Step 2: Arbitrarily select a wrongly-classified test point, z_test
# Step 3: Compute the influence I_up,loss(z, z_test) for every training point z
# Step 4: Select the 500 training points with the largest |I_up,loss(z, z_test)|
# Step 5: Compute the actual change in test loss after removing the point and
#         retraining for each of the 500 points
# Step 6: Plot -1/n (I_up,loss(z, z_test)) vs. actual change in test loss for
#         each of the 500 points

In [3]:
# Step 1: Train a model on the 10-class MNIST dataset

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [8]:
# define model architecture
model = nn.Sequential(
    nn.Linear(784, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# define loss function
criterion = nn.CrossEntropyLoss()

In [10]:
# training, validation data
train_data = torchvision.datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
train, val = random_split(train_data, [55000, 5000])

# data loaders
train_loader = DataLoader(train, batch_size=32)
val_loader = DataLoader(val, batch_size=32)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 5416203.00it/s] 


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 157696.12it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1498680.70it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3967207.16it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [11]:
# training loops
epochs = 5
for epoch in range(epochs):
    losses = list()
    for batch in train_loader:
        x, y = batch
        x = x.view(x.size(0), -1) # x.shape == [batch_size, 784]

        y_hat = model(x)
        loss = criterion(y_hat, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    print(f'Epoch: {epoch+1}, train loss: {torch.tensor(losses).mean():.2f}')


Epoch: 1, train loss: 0.33
Epoch: 2, train loss: 0.15
Epoch: 3, train loss: 0.11
Epoch: 4, train loss: 0.08
Epoch: 5, train loss: 0.07


In [12]:
# training loops

losses = list()
for batch in val_loader:
    x, y = batch
    x = x.view(x.size(0), -1) # x.shape == [batch_size, 784]

    with torch.no_grad():
      y_hat = model(x)
    loss = criterion(y_hat, y)
    losses.append(loss.item())
print(f'val loss: {torch.tensor(losses).mean():.2f}')

val loss: 0.13
